In [169]:
import pandas as pd

In [170]:
df=pd.read_csv("../output/input_2021-01-01.csv", usecols=["patient_id", "bmi", "weight", "height", "age", "sex", "serum_creatinine", "atrial_fib", "doac_dose_calculated"])
#calculate actual, ideal and adjusted weights for all patients
df["ABW"] = df.weight
df.loc[df["sex"] == 'M', 'IBW'] = 50 + (2.3 * (df.height - 60))
df.loc[df["sex"] == 'F', 'IBW'] = 45.5 + (2.3 * (df.height - 60))
df["ADJ"] = df.IBW + 0.4 * (df.ABW - df.IBW)

#weight for calculation
df.loc[df["bmi"] <30, 'weight_crcl_1'] = df.ABW
df.loc[df["bmi"] >=30,'weight_crcl_1'] = df.IBW
df.loc[df["bmi"] ==0, 'weight_crcl_1'] = 0

df.loc[df["bmi"] <30, 'weight_crcl_2'] = df.ABW
df.loc[df["bmi"] >=30,'weight_crcl_2'] = df.ADJ
df.loc[df["bmi"] ==0, 'weight_crcl_2'] = 0

#crcl range
df.loc[df["sex"] == 'M', 'crcl_1'] = (1.23 * (140-df.age) * df.weight_crcl_1) / df.serum_creatinine
df.loc[df["sex"] == 'F', 'crcl_1'] = (1.04 * (140-df.age) * df.weight_crcl_1) / df.serum_creatinine

df.loc[df["sex"] == 'M', 'crcl_2'] = (1.23 * (140-df.age) * df.weight_crcl_2) / df.serum_creatinine
df.loc[df["sex"] == 'F', 'crcl_2'] = (1.04 * (140-df.age) * df.weight_crcl_2) / df.serum_creatinine

#apixaban
df.loc[(df["atrial_fib"] == 1) & (df["crcl_1"] <15), 'apixaban'] = 'nr'

df.loc[(df["atrial_fib"] == 1) & (df["crcl_1"] >=15) & (df["crcl_2"] <=29), 'apixaban'] = 'A2.5'

df.loc[(df["atrial_fib"] == 1) & (df["crcl_1"] >=30) & (df["age"] >=80) & (df["weight"] >60) & (df["serum_creatinine"] <133), 'apixaban'] = 'A5'
df.loc[(df["atrial_fib"] == 1) & (df["crcl_1"] >=30) & (df["weight"] <=60) & (df["age"] <80) & (df["serum_creatinine"] <133), 'apixaban'] = 'A5'
df.loc[(df["atrial_fib"] == 1) & (df["crcl_1"] >=30) & (df["serum_creatinine"] >=133) & (df["weight"] >60) & (df["age"] <80), 'apixaban'] = 'A5'

df.loc[(df["atrial_fib"] == 1) & (df["crcl_1"] >=30) & (df["age"] >=80) & (df["weight"] <=60) & (df["serum_creatinine"] <133), 'apixaban'] = 'A2.5'
df.loc[(df["atrial_fib"] == 1) & (df["crcl_1"] >=30) & (df["weight"] <=60) & (df["age"] <80) & (df["serum_creatinine"] >=133), 'apixaban'] = 'A2.5'
df.loc[(df["atrial_fib"] == 1) & (df["crcl_1"] >=30) & (df["serum_creatinine"] >=133) & (df["weight"] <=60) & (df["age"] <80), 'apixaban'] = 'A2.5'

#rivaroxaban
df.loc[(df["atrial_fib"] == 1) & df["crcl_1"] <15, 'rivaroxaban'] = 'nr'

df.loc[(df["atrial_fib"] == 1) & (df["crcl_1"] >=15) & (df["crcl_2"] <=49), 'rivaroxaban'] = 'R15'

df.loc[(df["atrial_fib"] == 1) & (df["crcl_1"] >50), 'rivaroxaban'] = 'R20'

#edoxaban
df.loc[(df["atrial_fib"] == 1) & df["crcl_1"] <15, 'edoxaban'] = 'nr'

df.loc[(df["atrial_fib"] == 1) & (df["crcl_1"] >=15) & (df["crcl_2"] <=50), 'edoxaban'] = 'E30'
df.loc[(df["atrial_fib"] == 1) & (df["crcl_1"] >50) & (df["weight"] <=60), 'edoxaban'] = 'E30' ### + is on certain p-gp inhibitors ciclosporin, dronedarone, erthromycin, ketoconazole

df.loc[(df["atrial_fib"] == 1) & (df["crcl_1"] >50) & (df["weight"] >60), 'edoxaban'] = 'E60' ### + not on certain p-gp inhibitors ciclosporin, dronedarone, erthromycin, ketoconazole










df.head(50)


,doac_dose_calculated,atrial_fib,serum_creatinine,bmi,weight,height,age,sex,patient_id,ABW,IBW,ADJ,weight_crcl_1,weight_crcl_2,crcl_1,crcl_2,apixaban,rivaroxaban
0,NaN,0,54.321029,12.727464,25.421232,71.068010,37,M,66368,25.421232,75.456423,55.442347,25.421232,25.421232,59.288567,59.288567,NaN,nr
1,R15/20 titration,0,67.743407,28.870268,58.731607,51.621019,50,M,60796,58.731607,30.728343,41.929649,58.731607,58.731607,95.973752,95.973752,NaN,nr
2,NaN,0,43.011732,36.114236,37.093741,73.970994,73,F,87376,37.093741,77.633286,61.417468,77.633286,61.417468,125.767716,99.497717,NaN,nr
3,NaN,0,39.343818,24.159660,60.707662,50.647395,46,M,61642,60.707662,28.489009,41.376471,60.707662,60.707662,178.402105,178.402105,NaN,nr
4,NaN,1,82.217671,32.967184,0.000000,74.521562,88,F,80500,0.000000,78.899592,47.339755,78.899592,47.339755,51.897480,31.138488,A2.5,R20
5,NaN,0,45.956547,28.506081,64.636714,47.174278,71,F,49062,64.636714,16.000839,35.455189,64.636714,64.636714,100.928615,100.928615,NaN,nr
6,R10,0,63.584763,0.000000,86.351486,74.715922,59,F,20879,86.351486,79.346620,82.148567,0.000000,0.000000,0.000000,0.000000,NaN,nr
7,NaN,0,64.845477,25.443140,70.718404,80.272447,12,F,22204,70.718404,92.126627,83.563338,70.718404,70.718404,145.176415,145.176415,NaN,nr
8,NaN,0,66.147061,26.414141,60.861256,77.206750,57,M,71518,60.861256,89.575524,78.089817,60.861256,60.861256,93.931998,93.931998,NaN,nr
9,NaN,0,49.658045,25.712368,54.098372,0.000000,42,M,94482,54.098372,-88.000000,-31.160651,54.098372,54.098372,131.318456,131.318456,NaN,nr
